<a href="https://colab.research.google.com/github/nathalyAlarconT/GenAI_Workshops/blob/main/MultiAgents_VertexAI_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install --upgrade --quiet  langchain-core langchain-google-vertexai
!pip install --quiet crewai crewai-tools
!pip install --quiet google-cloud-aiplatform

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

## Vertex AI - Authentication

In [ ]:
from langchain_google_vertexai import VertexAI
import vertexai
import json
import os

In [ ]:
# Let's authenticate our user
from google.colab import auth as google_auth
google_auth.authenticate_user()

In [ ]:
from google.colab import userdata

# Replace here your GCP PROJECT NAME and PROJECT LOCATION
GCP_PROJECT = userdata.get('GCP_Project')
GCP_PROJECT_LOCATION = userdata.get('GCP_Project_location')

vertexai.init(project=GCP_PROJECT, location=GCP_PROJECT_LOCATION)

In [ ]:
gcp_vertex_ai_credentials = json.loads(userdata.get('GCP_Service_Account'))

# Crew AI Setup

In [ ]:
from crewai import Agent, Task, Crew, LLM
from crewai_tools import SerperDevTool

To use the Search tool, we need to add our Serper Search API KEY
You can create your free account here:
https://serper.dev/api-key

In [ ]:
os.environ['SERPER_API_KEY'] = userdata.get('Serper_Search_APIKEY')

## Gemini Model Configuration

In [ ]:
gemini_llm = LLM(
              model='vertex_ai/gemini-2.0-flash',
              vertex_credentials = gcp_vertex_ai_credentials,
              vertex_project = GCP_PROJECT
            )

## Tools Definition

In [ ]:
google_search_tool = SerperDevTool()

## Agents Definition

In [ ]:
research_agent = Agent(
    role="Senior Research",
	  goal="Get the most important facts about {topic} ,"
          "organize the information, so it can be used it later to "
          "write a blog post about it.",
	  backstory=("You work at {company_name} in the Research Department"
          " Now you are working in the research process of {topic} "
          " You need to make sure that you are not making assumptions"
          " and that you are providing the most accurate information."),
    llm = gemini_llm,
    allow_delegation=False,
    max_iterations=2,
    verbose=True
)

In [ ]:
writer_agent = Agent(
    role="Senior Writer",
	  goal="You have to write a blog post about {topic} ,"
         "You will be provided with all the important facts about {topic} ,"
         "The blog needs to be written in a way that is easy to read and understand."
         "Validate that the data is correct and make no assumptions",
	  backstory=("You work at {company_name} in the Writing Department"
          " Now you are working in the research process of {topic} "
          " You are recognized because your articles are the ones with more engagement"),
    llm = gemini_llm,
    allow_delegation=False,
    max_iterations=2,
    verbose=True
)

In [ ]:
marketing_agent = Agent(
    role="Senior Marketing",
	  goal="You have to write a marketer post to promote the blog post about {topic} ,"
         "Use the most interesting parts of the blog post and add a "
         "call to action to the company site of {company_name}" ,
    backstory = "You work at {company_name} in the Marketing Department"
          " Your team will write and marketer post of  {topic} "
          " You manage: Facebook , Linkedin and X",

    llm = gemini_llm,
    allow_delegation=False,
    max_iterations=2,
    verbose=True
)

In [ ]:
agency_supervisor_agent = Agent(
    role="Agency Supervisor",
	  goal="Make sure that your company : {company_name} "
          "Delivers a great blog article about {topic}" ,
    backstory = "You are a great leader of the agency for company {company_name}"
                "You have a great team that delivers quality blog posts",

    llm = gemini_llm,
    allow_delegation=True,
    max_iterations=2,
    verbose=True
)


## Tasks Definition

In [ ]:
research_about_topic = Task(
    description=("Review among the public knowledge available about {topic}, "
                "get the most interesting points and prepare an initial document "
                "organized by sections "
                "with the most interesting facts about {topic}"
    ),
    expected_output=(
        "A detailed document with the most interesting topics about {topic} "
        "organized by sections. Include also a sections for: "
        "Origin, Interesting facts, Current implementations,  Future Steps"
        "Use markdown to separate sections."
    ),
	  tools=[google_search_tool],
    agent=research_agent,
    output_file = "Research.md"
)

In [ ]:
write_about_topic = Task(
    description=("Write a blog post about {topic} , "
                " Do not include complex terminology"),
    expected_output=("A blog post about {topic}. The post should have 4 paragraphs. "
                      "Start with an open question.  Then describe the main concept on {topic}"
                      "Show the most interesting points. "
                      "Close the blog post creating expectation about the future Steps or projects of {topic}"
                      "Use markdown for the final blog post"),
    agent=writer_agent,
    output_file = "Blog_post.md"
)


In [ ]:
write_for_media_topic = Task(
    description=("Write a post material to promote the blog about: {topic} , "
                " Do not include complex terminology"),
    expected_output=("Content to promote the  {topic}. "
                     "Create a post for: Linkedin, X and Facebook"),
    agent=marketing_agent,
    output_file = "Marketing_post.md"
)


## Crew Setup

In [ ]:
crew = Crew(
  agents=[research_agent, writer_agent, marketing_agent, agency_supervisor_agent],
  tasks=[research_about_topic, write_about_topic, write_for_media_topic],
  verbose=True,
  llm=gemini_llm
)

# Crew Execution

In [ ]:
inputs = {
    "company_name": "The Explainers",
    "topic": "AI in Latin America"
    }
result = crew.kickoff(inputs=inputs)

# Agent: Senior Research
## Task: Review among the public knowledge available about AI in Latin America, get the most interesting points and prepare an initial document organized by sections with the most interesting facts about AI in Latin America


# Agent: Senior Research
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"AI in Latin America origin\"}"
## Tool Output: 
{'searchParameters': {'q': 'AI in Latin America origin', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Latin American Artificial Intelligence Index (ILIA) Reconfirms Chile ...', 'link': 'http://www.cepal.org/en/pressreleases/latin-american-artificial-intelligence-index-ilia-reconfirms-chile-brazil-and-uruguay', 'snippet': 'Chile stands out as the regional leader in AI with high investment in technological infrastructure, training programs, and supporting policies.', 'position': 1}, {'title': 'The AI Revolution is Here: How Will Latin America and the ...', 'l

# Results

In [ ]:
from IPython.display import Markdown

## Final Blog Post

In [ ]:
Markdown("Blog_post.md")

```markdown
## Is Latin America Ready for the AI Revolution?

Artificial intelligence (AI) is rapidly changing the world, and Latin America is no exception. While the region may not always be the first place that comes to mind when thinking about tech innovation, many countries are actively exploring and implementing AI solutions to address local challenges and boost their economies. From Argentina and Colombia leading the charge with initial AI strategies, it's clear that Latin America is waking up to the potential of AI. This journey involves not just adopting existing technologies but also creating unique solutions tailored to the region's specific needs.

So, what makes AI in Latin America so interesting? Several factors stand out. Countries like Chile, Brazil, and Uruguay are leading the way in AI readiness and adoption. Brazil's AI market is projected to reach nearly 50 billion dollars by 2030! Governments are also getting involved; Uruguay, for instance, is focused on integrating AI into its public services in an ethical and transparent way. Plus, cities like Mexico City are seeing a boom in AI startups, fostering innovation and new ideas. AI is being used in diverse areas like healthcare for better diagnoses, finance for fraud detection, and even agriculture to improve farming practices. In fact, Latin America is adopting GenAI tools faster than advanced economies.

However, it's not all smooth sailing. The region faces challenges like the digital divide, ensuring everyone has access to technology and the skills to use it. There's also the risk of job displacement as AI automates certain tasks. It’s important to make sure that as AI advances, it benefits everyone and reduces inequality, rather than making it worse. Chile, for instance, is heavily investing in digital infrastructure, training programs, and policies to responsibly develop AI and has also launched its updated National AI Policy and action plan, as well as an AI bill.

What's next for AI in Latin America? Expect to see more comprehensive AI policies and regulations being developed across the region. There will be a greater emphasis on education and training programs to create a workforce ready for the AI era. Investment in digital infrastructure will continue, and ethical considerations surrounding AI will take center stage. Countries will likely collaborate more to share knowledge and address common challenges. The real magic will happen when AI solutions are tailored to the unique needs of Latin America, driving sustainable development, social inclusion, and economic growth. It's a journey filled with promise, and the world will be watching to see how Latin America shapes its AI future.
```

## Marketing For Blog Post

In [ ]:
Markdown('Marketing_post.md')

```
**LinkedIn Post:**

Headline: AI is Booming in Latin America! 🚀

Did you know Latin America is rapidly embracing AI? From Brazil's projected $49.2 billion AI market by 2030 to Uruguay's focus on ethical AI in public services, the region is full of innovation. Chile, Brazil, and Uruguay are leading the charge in AI readiness! Plus, Latin America is adopting GenAI faster than advanced economies!

But it's not without its challenges. Addressing the digital divide and ensuring equitable access are key.

Dive deeper into the exciting developments, challenges, and future of AI in Latin America in our latest blog post! [Link to The Explainers Blog Post] #AI #LatinAmerica #ArtificialIntelligence #Innovation #EmergingTech #TheExplainers

**X (Twitter) Post:**

AI in Latin America is heating up! 🔥 Brazil's AI market could hit $49.2B by 2030! Uruguay's using AI ethically in government. Latin America is adopting GenAI faster than advanced economies. What's next?

Learn more about the rise of AI in LATAM & the challenges ahead: [Link to The Explainers Blog Post] #AI #LatinAmerica #Innovation #Tech

**Facebook Post:**

Image/Video Suggestion: An image featuring a collage of Latin American cityscapes with AI/tech-related icons overlaid, or a short video highlighting key stats from the blog post.

Text:
Latin America is becoming a hotbed for Artificial Intelligence! 🤖 From healthcare advancements to fintech innovation and impressive GenAI adoption, AI is transforming industries across the region. Countries like Chile, Brazil, and Uruguay are at the forefront, with exciting government initiatives and a growing startup scene.

Did you know Brazil's AI market is expected to reach nearly $50 BILLION by 2030?

However, there are important challenges to address, like bridging the digital divide and ensuring AI benefits everyone.

Want to know more about the incredible opportunities and hurdles facing AI in Latin America? Read our latest blog post! [Link to The Explainers Blog Post] #AIinLatinAmerica #ArtificialIntelligence #LATAMTech #Innovation #DigitalTransformation #TheExplainers
```

## Initial Research

In [ ]:
Markdown('Research.md')

```markdown
# AI in Latin America: An Overview

## Origin

The initial development of AI strategies in Latin America can be traced back to 2019, with Argentina and Colombia being among the first countries to publish standalone AI strategies. However, the broader history involves a gradual increase in awareness and adoption, influenced by global trends and specific regional needs. The region has recognized AI's potential to transform various sectors and address local challenges, leading to increased investment and policy development.

## Interesting Facts

*   **Regional Leadership:** Chile, Brazil, and Uruguay consistently emerge as leaders in AI readiness and adoption in Latin America.
*   **Investment and Growth:** The AI market in Brazil is projected to reach USD 49.2 billion by 2030, reflecting significant growth potential in the region.
*   **Government Initiatives:** Uruguay stands out for its focus on integrating AI into public administration with an emphasis on ethical and transparent use.
*   **Startup Ecosystem:** Mexico boasts a growing AI startup ecosystem, particularly in Mexico City, indicating increasing innovation and entrepreneurial activity.
*   **Cross-sector Applications:** AI is being applied across diverse sectors, from healthcare and finance to agriculture and government services, demonstrating its versatility and potential impact.
*   **Digital Divide and Inequality:** While AI presents numerous opportunities, challenges related to data access, digital literacy, and potential job displacement need to be addressed to ensure inclusive and equitable development.
*   **GenAI Adoption**: Latin America surpasses advanced economies in ChatGPT and Generative AI adoption

## Current Implementations

### Chile

*   **Open-Format Tools:** Development of open-format AI tools and language models tailored to the region's specific needs.
*   **Digital Transformation:** Government initiatives focused on fostering digital transformation through responsible data use and AI systems.
*   **Infrastructure Investment:** High investment in technological infrastructure, training programs, and supporting policies for AI development.
*   **National AI Policy:** Chile has launched its updated National AI Policy and action plan, as well as an AI bill aimed at regulating and promoting the ethical and responsible.

### Brazil

*   **Healthcare Applications:** AI systems used for medical diagnosis, image analysis, and disease prediction.
*   **MSME Adoption:** Micro, small, and medium enterprises (MSMEs) are leveraging AI to improve customer experience, increase efficiency, and drive business development.
*   **Fintech Innovation**: Brazilian fintechs are using generative AI to improve their services.

### Uruguay

*   **Public Administration Integration:** Focus on integrating AI into public administration with an emphasis on ethical and transparent use.
*   **AI Ethics and Governance:** Strong AI legal framework and data protection measures, with ongoing revisions to its AI strategy.
*   **Tax Fraud Detection**: Use of AI systems to detect tax fraud.
*   **E-government strategy:** Developing processes and defining standards for AI strategy across the country.

### Mexico

*   **Urban Applications:** AI used for mobility, health, and government communication in cities like Jalisco, Mexico City, and Monterrey.
*   **Startup Growth:** A vibrant AI startup ecosystem, especially in Mexico City.
*   **Policy Making:** AI is used for evidence-based policy-making, as an aid in certain trials, and in the healthcare.

## Future Steps

*   **Policy and Regulation:** Development of comprehensive AI policies and regulatory frameworks to promote responsible AI development and deployment.
*   **Skills Development:** Investment in education and training programs to build a skilled workforce capable of developing, deploying, and managing AI technologies.
*   **Infrastructure Development:** Continued investment in digital infrastructure, including high-speed internet and data centers, to support AI innovation.
*   **Ethical Considerations:** Addressing ethical considerations related to data privacy, algorithmic bias, and potential job displacement.
*   **Regional Collaboration:** Fostering collaboration among countries in Latin America to share best practices, address common challenges, and promote regional AI initiatives.
*   **Focus on Local Needs:** Tailoring AI solutions to address specific challenges and opportunities in the region, such as sustainable development, social inclusion, and economic growth.
*   **Promote AI Safety**: opportunities for ethical and human rights-focused governance.
```

 Google Cloud credits are provided for this project.

 #VertexAISprint